**Face Mask Detector using Python, OpenCV**

**Asad Tariq**
**19I-0659**

In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import copy as cp
import matplotlib.pyplot as plt
import os

In [2]:
# Checking Colors for Mask
def check_color(face_mask_img):
    count_reds = 0
    count_blue = 0
    count_green = 0
    count_black = 0

    imgH, imgW, channel = face_mask_img.shape
    for i in range(0, imgH):
        for j in range(0, imgW):
            if face_mask_img[i][j][2] > face_mask_img[i][j][1] and face_mask_img[i][j][1] > face_mask_img[i][j][0]:
                count_reds += 1

            elif face_mask_img[i][j][0] > face_mask_img[i][j][1] and face_mask_img[i][j][1] > face_mask_img[i][j][2]:
                count_blue += 1

            elif face_mask_img[i][j][1] > face_mask_img[i][j][0] and face_mask_img[i][j][0] > face_mask_img[i][j][2]:
                count_green += 1

            elif face_mask_img[i][j][0] <= 30 and face_mask_img[i][j][1] <= 30 and face_mask_img[i][j][2]:
                count_black += 1

    if count_reds > count_green and count_reds > count_blue and count_reds > count_black:
        # print("SKIN DETECTED !!!!")
        return 0

    if count_green > count_reds and count_green > count_blue and count_green > count_black:
        # print("MASK GREEN DETECTED !!!!")
        return 1

    if count_blue > count_green and count_blue > count_reds and count_blue > count_black:
        # print("MASK BLUE DETECTED !!!!")
        return 1

    if count_black > count_green and count_black > count_blue and count_black > count_reds:
        # print("MASK BLACK DETECTED !!!!")
        return 1


In [3]:
# Get Matching Result
def get_shape_match(face_mask_img_img, cont, index_value, modelImage):
    face_mask_img = cp.copy(face_mask_img_img)

    image_given = cv.cvtColor(face_mask_img, cv.COLOR_BGR2GRAY)

    ret, thresh_given_img = cv.threshold(image_given, 130, 255, cv.THRESH_BINARY)

    contours_given_image, hierarchy = cv.findContours(thresh_given_img, 2, 1)
    for contour_index in range(0, len(contours_given_image)):
        if cv.contourArea(contours_given_image[contour_index]) > 2000:
            result = cv.matchShapes(cont[index_value], contours_given_image[contour_index], 1, 0.0)
            # print("Contour Matching Value = " + str(result))
            if result <= 0.3:
                color_detection = check_color(face_mask_img_img)
                if color_detection == 1:
                    return 1

    return 0


In [4]:
# Function to crop Image
def get_image(img, min_x, min_y, max_x, max_y):
    imageHeight, imageWidth, _ = img.shape
    image2 = np.zeros((imageWidth, imageHeight, 3), np.uint8)
    if min_x >= 0 and min_y >= 0 and max_x <= imageWidth and max_y <= imageHeight:
        for ii in range(min_y, max_y):
            for jj in range(min_x, max_x):
                for kk in range(0, 3):
                    image2[ii][jj][kk] = img[ii][jj][kk]

        return image2


In [5]:
# Check Face Mask By Comparing With Models
def check_mask(face_mask_img):
    # read model images
    ret_x = -1
    total_model_images = 13
    for loop_index in range(1, total_model_images+1, 1):
        model_img_org = cv.imread("Model/model ("+str(loop_index)+").jpg")
        model_img = cv.cvtColor(model_img_org, cv.COLOR_BGR2GRAY)

        ret, thresh_img = cv.threshold(model_img, 130, 255, cv.THRESH_BINARY)
        contours, hierarchy = cv.findContours(thresh_img, 2, 1)

        if loop_index == 2 or loop_index == 3 or loop_index == 4 or loop_index == 6:
            ret_x = get_shape_match(face_mask_img, contours, 0, model_img_org)

        elif loop_index == 1 or 9 <= loop_index <= 13:
            ret_x = get_shape_match(face_mask_img, contours, 3, model_img_org)

        elif loop_index == 5:
            ret_x = get_shape_match(face_mask_img, contours, 1, model_img_org)

        elif loop_index == 8:
            ret_x = get_shape_match(face_mask_img, contours, 5, model_img_org)

        elif loop_index == 7:
            ret_x = get_shape_match(face_mask_img, contours, 6, model_img_org)

    return ret_x


In [25]:
# Using media pipe for landmarks
fm_face_mesh = mp.solutions.face_mesh
faceMesh = fm_face_mesh.FaceMesh()

# Including xml file for face detection
face_cascade = cv.CascadeClassifier('face_detection.xml')


In [26]:
def read_image(image_name:str):
    img = cv.imread(image_name)
    img = cv.resize(img, (256, 256))
    return img


In [52]:
def run_model(img_name:str):
    if type(img_name) != str:
        return TypeError("Type Must be of str")

    # Here we are detecting faces and cropping them
    img_original = read_image("Tests/" + img_name)
    img = cp.copy(img_original)

    img_height, img_width, _ = img.shape

    plt.imshow(cv.cvtColor(img, cv.COLOR_RGB2BGR))
    plt.show()

    gray_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_image, 1.3, 2)
    total_faces = len(faces)
    if total_faces:
        array = []
        cropping = []

        for i in range(total_faces):
            for (x, y, w, h) in faces:
                cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)   # Draw Rectangle Around Face
                array.append(img)
                crop = array[i][y:y + h, x:x + w]
                cropping.append(crop)

        # For drawing bounding rectangle around the face mask area
        min_x = -1
        max_x = -1
        min_y = -1
        max_y = -1

        index = 1  # It will count the images and save them by number then I used it for deleting the files
        for a in range(total_faces):
            cv.imwrite("CropImage" + str(index) + ".jpg", cropping[a])
            img = cv.imread("CropImage" + str(index) + ".jpg")
            img_height, img_width, _ = img.shape
            rgb_image = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            landmarks_result = faceMesh.process(rgb_image)
            if landmarks_result.multi_face_landmarks:
                for facial_landmarks in landmarks_result.multi_face_landmarks:
                    for landmarks_index in range(0, 468):
                        points = facial_landmarks.landmark[landmarks_index]
                        x_point = int(points.x * img_width)
                        y_point = int(points.y * img_height)
                        if y_point > int((img_height / 2) - 10):

                            # calculate min max vertical point
                            if min_y == -1:
                                min_y = y_point
                            elif y_point < min_y:
                                min_y = y_point

                            if max_y == -1:
                                max_y = y_point
                            elif y_point > max_y:
                                max_y = y_point

                            # calculate min max horizontal point
                            if min_x == -1:
                                min_x = x_point
                            elif x_point < min_x:
                                min_x = x_point

                            if max_x == -1:
                                max_x = x_point
                            elif x_point > max_x:
                                max_x = x_point

                        else:
                            # cv.circle(img, (x_point, y_point), 1, (255, 0, 0), -1)
                            pass

                img_face_mask_area = get_image(img, min_x, min_y, max_x, max_y)

                mask = -100
                try:
                    mask = check_mask(img_face_mask_area)

                except:
                    print("Error in Check Mask")

                if mask == 1:
                    cv.putText(img_original, "FACE MASK", (10, 30), cv.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
                else:
                    cv.putText(img_original, "NO FACE", (10, 30), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
                    cv.putText(img_original, "MASK", (20, 50), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)

                plt.imshow(cv.cvtColor(img_original, cv.COLOR_RGB2BGR))
                plt.show()

                plt.imshow(cv.cvtColor(img, cv.COLOR_RGB2BGR))
                plt.show()
                index += 1

                min_x = -1
                min_y = -1
                max_x = -1
                max_y = -1

        for i in range(1, index):
            try:
                os.remove("CropImage" + str(i) + ".jpg")

            except:
                print("Error in Deleting Files")
    else:
        print(total_faces)
        print("Face Detection Failed !!!")


In [ ]:
run_model("test1.jpg")